<a href="https://colab.research.google.com/github/haizzzi/simple-nmt-simplied/blob/main/NTC_Exe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# GPU가 있는지 확인
if torch.cuda.is_available():
    print("사용 가능한 GPU 수:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i} 이름:", torch.cuda.get_device_name(i))
else:
    print("GPU를 사용할 수 없습니다.")


사용 가능한 GPU 수: 1
GPU 0 이름: Tesla T4


In [4]:
!pip install torch torchvision torchaudio
!pip install torch_optimizer
!pip install pytorch-ignite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [1]:
!wc -l /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv

147893 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv


이 코드는 train과 valid 나누는 코드인데 불필요함. train만 넣으면 학습 코드에서 자동으로 나눠줌

In [2]:
!head -n 118314 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv > /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train1.tsv
!tail -n 29579 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv > /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.valid1.tsv


In [3]:
!wc -l /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.*.tsv

   30000 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.test.tsv
  118314 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train1.tsv
  147893 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv
   29579 /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.valid1.tsv
  325786 total


simple-ntc 학습하는 코드, rnn, cnn 같이 학습함. 이후 classify할 때 앙상블로 두개 모두 활용함.



In [5]:
!python /content/drive/MyDrive/nmt_fold/simple-ntc/train.py --model_fn /content/drive/MyDrive/nmt_fold/data2/review.pth --train_fn /content/drive/MyDrive/nmt_fold/data2/review.sorted.uniq.refined.tok.shuf.train.tsv --gpu_id 0 --batch_size 128 --n_epochs 10 --word_vec_size 256 --dropout .3 --rnn --hidden_size 512 --n_layers 4 --cnn --window_sizes 3 4 5 6 7 8 --n_filters 128 128 128 128 128 128


|train| = 118315 |valid| = 29578
|vocab| = 13048 |classes| = 2
RNNClassifier(
  (emb): Embedding(13048, 256)
  (rnn): LSTM(256, 512, num_layers=4, batch_first=True, dropout=0.3, bidirectional=True)
  (generator): Linear(in_features=1024, out_features=2, bias=True)
  (activation): LogSoftmax(dim=-1)
)
Epoch 1 - |param|=1.83e+03 |g_param|=1.02e-01 loss=6.8568e-01 accuracy=0.5613
Validation - loss=6.8691e-01 accuracy=0.5538 best_loss=inf
Epoch 2 - |param|=1.84e+03 |g_param|=3.25e-01 loss=2.3030e-01 accuracy=0.9182
Validation - loss=2.1659e-01 accuracy=0.9235 best_loss=6.8691e-01
Epoch 3 - |param|=1.84e+03 |g_param|=4.22e+00 loss=1.8298e-01 accuracy=0.9385
Validation - loss=1.9748e-01 accuracy=0.9313 best_loss=2.1659e-01
Epoch 4 - |param|=1.85e+03 |g_param|=2.83e-01 loss=1.7139e-01 accuracy=0.9439
Validation - loss=1.9031e-01 accuracy=0.9323 best_loss=1.9748e-01
Epoch [5/10]:  15%|▉     | 136/925 [02:44<17:12,  1.31s/it, loss=0.14, accuracy=0.955, |param|=1.85e+3, |g_param|=0.267]Engine ru